# Use Python function to predict business area and action for car rental company with `ibm-watson-machine-learning`

AI function with logic to sequentially call two models: BUSINESS_AREA and ACTION.

You will use **car_rental_training** dataset.


## Learning goals

The learning goals of this notebook are:

-  AI function definition
-  Store AI function
-  Deployment creation


## Contents

This notebook contains the following parts:
1. [Setup](#setup)
2. [Define AI function](#ai_function)
3. [Store the AI function in the repository](#upload)
4. [Deploy AI function](#deploy)
5. [Clean up](#cleanup)
6. [Summary](#summary)

<a id="setup"></a>
## 1. Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Contact with your Cloud Pack for Data administrator and ask him for your account credentials

### Connection to WML

Authenticate the Watson Machine Learning service on IBM Cloud Pack for Data. You need to provide platform `url`, your `username` and `api_key`.

In [ ]:
username = 'PASTE YOUR USERNAME HERE'
api_key = 'PASTE YOUR API_KEY HERE'
url = 'PASTE THE PLATFORM URL HERE'

In [2]:
wml_credentials = {
    "username": username,
    "apikey": api_key,
    "url": url,
    "instance_id": 'openshift',
    "version": '4.6'
}

Alternatively you can use `username` and `password` to authenticate WML services.

```
wml_credentials = {
    "username": ***,
    "password": ***,
    "url": ***,
    "instance_id": 'openshift',
    "version": '4.6'
}

```

### Install and import the `ibm-watson-machine-learning` package
**Note:** `ibm-watson-machine-learning` documentation can be found <a href="http://ibm-wml-api-pyclient.mybluemix.net/" target="_blank" rel="noopener no referrer">here</a>.

In [ ]:
!pip install -U ibm-watson-machine-learning

In [3]:
from ibm_watson_machine_learning import APIClient

client = APIClient(wml_credentials)

### Working with spaces

First of all, you need to create a space that will be used for your work. If you do not have space already created, you can use `{PLATFORM_URL}/ml-runtime/spaces?context=icp4data` to create one.

- Click New Deployment Space
- Create an empty space
- Go to space `Settings` tab
- Copy `space_id` and paste it below

**Tip**: You can also use SDK to prepare the space for your work. More information can be found [here](https://github.com/IBM/watson-machine-learning-samples/blob/master/cpd4.5/notebooks/python_sdk/instance-management/Space%20management.ipynb).

**Action**: Assign space ID below

In [ ]:
space_id = 'PASTE YOUR SPACE ID HERE'

You can use `list` method to print all existing spaces.

In [ ]:
client.spaces.list(limit=10)

To be able to interact with all resources available in Watson Machine Learning, you need to set **space** which you will be using.

In [4]:
client.set.default_space(space_id)

'SUCCESS'

<a id="ai_function"></a>
## 2. Define AI function

In this section we define AI function and test it locally.

### Template

We start with template version of AI function to present its structure in simplest case.

    # Template version

    def score_template(payload):
        """AI function example.

        # Here we present data schema to be used by function:
        Example:
        {
          "input_data":
              [{
                "fields": ["Gender", "Status", "Children", "Age", "Customer_Status"],
                "values": [
                  ["Male", "M", 2, 48, "Inactive"],
                  ["Female", "S", 0, 23, "Inactive"]
                ]
              }]
        }
        """
        fields = payload['input_data'][0]['fields'] + ['Prediction', 'Probability']
        values = [record + [int(record[0] == 'Male'), 0.9] for record in payload['input_data'][0]['values']]
        return { 'fields': fields, 'values': values }

    template_payload = {'input_data': [{"fields": [],
                        "values": [["Male", "M", 2, 48, "Inactive"], ["Female", "S", 0, 23, "Inactive"]]}]}

    print(score_template(template_payload))

### Deployed models metadata extraction

We have to identify model deployments to be used before we define AI function. We extract scoring url and information about model version.

We list all deployed models predicting `Action` and `Business Area` to select one to be used in the AI function.

In [ ]:
client.deployments.list()

We select deployed model to predict `Business Area` and model to recommend `Action`. We have to extract deployments IDs.

**Tip**: If you don't have required deployments ready yet, please refer to the following notebooks to prepare them:
- [Bisnes area](https://github.com/IBM/watson-machine-learning-samples/blob/master/cpd4.5/notebooks/python_sdk/deployments/spark/cars-4-you/Use%20Spark%20to%20predict%20business%20area%20for%20car%20rental%20company.ipynb)
- [Action Recommendation](https://github.com/IBM/watson-machine-learning-samples/blob/master/cpd4.5/notebooks/python_sdk/deployments/spark/cars-4-you/Use%20Spark%20to%20recommend%20mitigation%20for%20car%20rental%20company.ipynb)

In [5]:
deployments = client.deployments.get_details()['resources']

for x in deployments:
    if (x['entity']['name'] == 'CARS4U - Action Recommendation model deployment'):
        action_deployment_id = x['metadata']['id']
    if (x['entity']['name'] == 'CARS4U - Business Area Prediction Model deployment'):
        business_area_deployment_id = x['metadata']['id']
        
print(f"Action deployment id: {action_deployment_id}")
print(f"Business area deployment id: {business_area_deployment_id}")

Action deployment id: 87a4973b-7b66-49c5-bd51-d1493c2681a9
Business area deployment id: dcb4d6d3-4165-49ad-bca6-9c6d27d9ae71


### AI function definition

We define AI function to be deployed in the IBM Cloud.

Put all parameters required to define AI function in dictionary.

In [6]:
ai_params = {"wml_credentials": wml_credentials, 
             "space_id": space_id,
             "action_deployment_id": action_deployment_id, 
             "business_area_deployment_id": business_area_deployment_id}

In [7]:
def score_generator(params=ai_params):
    
    from ibm_watson_machine_learning import APIClient

    wml_credentials = params["wml_credentials"]
    space_id = params['space_id']
    action_deployment_id = params["action_deployment_id"]
    business_area_deployment_id = params["business_area_deployment_id"]

    client = APIClient(wml_credentials)
    client.set.default_space(space_id)

    def score(payload):
        """AI function with model version.

        Example:
          {"fields": ["ID", "Gender", "Status", "Children", "Age", "Customer_Status", "Car_Owner", "Customer_Service", "Satisfaction"],
           "values": [[2624, 'Male', 'S', 0, 49.27, 'Active', 'No', "Good experience with all the rental co.'s I contacted. I Just called with rental dates and received pricing and selected rental co.", 1]]}
        """
        
        fields = payload["input_data"][0]['fields']
        
        scores_area = client.deployments.score(business_area_deployment_id, payload).get('predictions', [])
        idx_area = [i for i, x in enumerate(scores_area[0]["fields"]) if (x == 'probability') or (x == 'predictedLabel')]

        fields_from_action = fields + ["predictedLabel"]
        
        idx_from_action = set([i for i, x in enumerate(scores_area[0]["fields"]) if x in fields_from_action])
        values_action = [[v for i, v in enumerate(rec) if i in idx_from_action] for rec in scores_area[0]["values"]]
        
        payload_action = {
            "input_data": [
                {
                    "fields": fields + ["Business_Area"], 
                    "values": values_action
                }
            ]
        }
        
        scores_action = client.deployments.score(action_deployment_id, payload_action).get('predictions', [])
        idx_action = [i for i, x in enumerate(scores_action[0]["fields"]) if (x == 'probability') or (x == 'predictedLabel')]

        values = [rec + [scores_area[0]['values'][i][idx_area[0]], scores_area[0]['values'][i][idx_area[1]], scores_action[0]['values'][i][idx_action[0]], scores_action[0]['values'][i][idx_action[1]]] for i, rec in enumerate(payload["input_data"][0]['values'])]

        fields = fields + ['Probability_Area', 'Prediction_Area'] + ['Probability_Action', 'Prediction_Action']

        return {'predictions': [{'fields': fields, 'values': values}]}

    return score

### AI function local test

Let's test defined AI function in notebook before we deploy it in the IBM Cloud. We start with preparing sample payload for `score` function.

In [8]:
sample_payload = {
    "input_data": [
        {
            "fields": ["ID", "Gender", "Status", "Children", "Age", "Customer_Status", "Car_Owner", "Customer_Service", "Satisfaction"],
            "values": [[2624, 'Male', 'S', 0, 49.27, 'Active', 'No', "Good experience with all the rental co.'s I contacted. I Just called with rental dates and received pricing and selected rental co.", 1]]
        }       
    ]
}

Let's test AI function locally before we deploy it to the IBM Cloud.

In [9]:
score = score_generator()
scores_ai = score(sample_payload)
print(scores_ai)

{'predictions': [{'fields': ['ID', 'Gender', 'Status', 'Children', 'Age', 'Customer_Status', 'Car_Owner', 'Customer_Service', 'Satisfaction', 'Probability_Area', 'Prediction_Area', 'Probability_Action', 'Prediction_Action'], 'values': [[2624, 'Male', 'S', 0, 49.27, 'Active', 'No', "Good experience with all the rental co.'s I contacted. I Just called with rental dates and received pricing and selected rental co.", 1, [0.4711864406779661, 0.2745762711864407, 0.04067796610169491, 0.05423728813559322, 0.04406779661016949, 0.04745762711864407, 0.04406779661016949, 0.023728813559322035], 'Service: Knowledge', [1.0, 0.0, 0.0, 0.0, 0.0], 'NA']]}]}


<a id="upload"></a>
## 3. Store the AI function in the repository

In this section we store AI function to Watson machine Learning repository.

Required prerequisite to store AI function.

In [10]:
function_meta_props = {
            client.repository.FunctionMetaNames.NAME: 'CARS4U - Business area and Action Prediction - AI Function',
            client.repository.FunctionMetaNames.SOFTWARE_SPEC_ID: client.software_specifications.get_id_by_name("runtime-22.2-py3.10")
}

In [11]:
function_details = client.repository.store_function(score_generator, function_meta_props)

In [12]:
published_function_id = client.repository.get_function_id(function_details)

print("Function Id: " + str(published_function_id))

Function Id: f81ac85b-2cc1-4301-a92c-951135c284f5


In [ ]:
client.repository.get_function_details(published_function_id)

<a id="deploy"></a>
## 4. Deploy AI function

In this section we deploy AI function in the IBM Cloud and test created deployment using sample payload.

In [ ]:
client.deployments.list()

In [14]:
deployment_details = client.deployments.create(
    published_function_id, 
    meta_props={
        client.deployments.ConfigurationMetaNames.NAME: "CARS4U - Business area and Action Prediction - AI Function Deployment",
        client.deployments.ConfigurationMetaNames.ONLINE: {}
    }
)



#######################################################################################

Synchronous deployment creation for uid: 'f81ac85b-2cc1-4301-a92c-951135c284f5' started

#######################################################################################


initializing...
ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='07ff4ef1-5345-40eb-bae6-95cddbef4371'
------------------------------------------------------------------------------------------------




<a id="score"></a>
### AI function online deployment test

Now we are ready to test created deployment.

In [15]:
fields = ['ID', 'Gender', 'Status', 'Children', 'Age', 'Customer_Status','Car_Owner', 'Customer_Service', 'Satisfaction']
values = [3785, 'Male', 'S', 1, 17, 'Inactive', 'Yes', 'The car should have been brought to us instead of us trying to find it in the lot.', 0]

In [16]:
import json

payload_scoring = {"input_data": [{"fields": fields,"values": [values]}]}
scoring_response = client.deployments.score(deployment_details['metadata']['id'], payload_scoring)

print(json.dumps(scoring_response, indent=3))

{
   "predictions": [
      {
         "fields": [
            "ID",
            "Gender",
            "Status",
            "Children",
            "Age",
            "Customer_Status",
            "Car_Owner",
            "Customer_Service",
            "Satisfaction",
            "Probability_Area",
            "Prediction_Area",
            "Probability_Action",
            "Prediction_Action"
         ],
         "values": [
            [
               3785,
               "Male",
               "S",
               1,
               17,
               "Inactive",
               "Yes",
               "The car should have been brought to us instead of us trying to find it in the lot.",
               0,
               [
                  0.4711864406779661,
                  0.2745762711864407,
                  0.04067796610169491,
                  0.05423728813559322,
                  0.04406779661016949,
                  0.04745762711864407,
                  0.04406779661016

<a id="cleanup"></a>
## 5. Clean up 

If you want to clean up all created assets:
- experiments
- trainings
- pipelines
- model definitions
- models
- functions
- deployments

please follow up this sample [notebook](https://github.com/IBM/watson-machine-learning-samples/blob/master/cpd4.5/notebooks/python_sdk/instance-management/Machine%20Learning%20artifacts%20management.ipynb).

<a id="summary"></a>
## 6. Summary and next steps     

 You successfully completed this notebook! You learned how to use Apache Spark machine learning as well as Watson Machine Learning for model creation and deployment. 
 
Check out our [Online Documentation](https://dataplatform.cloud.ibm.com/docs/content/analyze-data/wml-setup.html) for more samples, tutorials, documentation, how-tos, and blog posts. 

### Authors

**Amadeusz Masny**, Python Software Developer in Watson Machine Learning at IBM

Copyright © 2020, 2021, 2022 IBM. This notebook and its source code are released under the terms of the MIT License.